In [7]:
import pandas as pd
import numpy as np
import os

In [8]:
os.getcwd()

'C:\\Users\\User\\OneDrive\\Desktop\\BI Developer Porfolio_Nawwarah\\06_Niche Project\\Supply Chain Analysis'

In [12]:
os.listdir("../Supply Chain Analysis/Data")

['.ipynb_checkpoints', 'raw_data.csv']

In [15]:
df = pd.read_csv("../Supply Chain Analysis/Data/raw_data.csv")

In [17]:
# 1. Feature Engineering: Lead Time

df['Order_Date'] = pd.to_datetime(df['Order_Date'])
df['Delivery_Date'] = pd.to_datetime(df['Delivery_Date'])
df['Actual_Lead_Time'] = (df['Delivery_Date'] - df['Order_Date']).dt.days

In [18]:
# 2. Calculating Supplier Volatility
# We group by Supplier to find their Mean and Std Dev of Lead Times

supplier_stats = df.groupby('Supplier_ID')['Actual_Lead_Time'].agg(['mean', 'std']).reset_index()

# Calculate Coefficient of Variation (CV)
# Handling division by zero with a small epsilon or fillna
supplier_stats['Volatility_Index'] = supplier_stats['std'] / supplier_stats['mean']
supplier_stats['Volatility_Index'] = supplier_stats['Volatility_Index'].fillna(0)

In [19]:
# 3. Assigning Risk Grades

def assign_risk(cv):
    if cv < 0.15: return 'Low'
    elif cv < 0.35: return 'Medium'
    else: return 'High'

supplier_stats['Risk_Grade'] = supplier_stats['Volatility_Index'].apply(assign_risk)

In [20]:
# Merge back to main dataframe

df_enriched = df.merge(supplier_stats[['Supplier_ID', 'Volatility_Index', 'Risk_Grade']], on='Supplier_ID')

In [21]:
# Export for SQL

df_enriched.to_csv('enriched_supply_chain_risk.csv', index=False)
print("Risk scores calculated and file exported for SQL!")

Risk scores calculated and file exported for SQL!
